# Here we construct KG from the data we have created

In [1]:
#import the libraries here

import numpy as np
import torch
import sklearn
import random
import pandas as pd

# Create the input data structure by reading the files

In [2]:
cd ..

C:\Prasad\MSIITM\Research\Dataset\train\pre_req_dataset


In [3]:
triplets = pd.read_csv('TrainingData.csv')
def process_triplets(number_of_triplets):
    # Initializing the dictionary to store the data
    triplets = number_of_triplets.sample(frac=1)
    data = []
    nodes = set(number_of_triplets['Head']) 
    nodes.update(set(number_of_triplets['Tail']))
    relations = set(number_of_triplets['Relation'])
    for index, row in triplets.iterrows():
        tmp = (row['Head'],row['Tail'],row['Relation'])
        data.append(tmp)
    return data, nodes, relations

# Process the triplets from the file
triplet_data, nodes, relations = process_triplets(triplets)

# Displaying a small portion of the data to verify correctness
print(len(triplet_data))


15259


In [4]:
import random

def split_data(data, train_percent=0.6, validation_percent=0.2, test_percent=0.2):
    # Shuffle the data to ensure random distribution
    random.shuffle(data)

    # Calculate the number of data points for each set
    total_data_points = len(data)
    train_size = int(total_data_points * train_percent)
    validation_size = int(total_data_points * validation_percent)

    # Split the data into training, validation, and test sets
    train_data = data[:train_size]
    validation_data = data[train_size:train_size + validation_size]
    test_data = data[train_size + validation_size:]

    return train_data, validation_data, test_data

In [5]:
import torch
# A function to convert our data into required format
def dataconv(data):
    # Extracting the columns
    heads = [row[0] for row in data]
    tails = [row[1] for row in data]
    relations = [row[2] for row in data]

    # Convert to PyTorch tensors
    head_tensor = torch.tensor(heads, dtype=torch.long)
    tail_tensor = torch.tensor(tails, dtype=torch.long)
    relation_tensor = torch.tensor(relations, dtype=torch.long)

    # Store in a dictionary
    graph_data = {
        'edge_index': torch.stack((head_tensor, tail_tensor), dim=0),
        'edge_reltype': relation_tensor,
        'num_nodes': 1268
    }
    return graph_data

In [6]:
train_set, val_set, test_set  = split_data(triplet_data)
train_set = dataconv(train_set)
val_set = dataconv(val_set)
test_set = dataconv(test_set)

In [7]:
print(len(train_set),len(val_set), len(test_set))

3 3 3


In [8]:
train_set.size()
# torch.randint(1269, 5 + 1269 , (1,))

AttributeError: 'dict' object has no attribute 'size'

# Now Dataset class and create negative edge for positive edge

In [9]:
from torch.utils.data import DataLoader, Dataset

class RelationDataset(Dataset):
  def __init__(self, true_edges, filter=False):
    self.true_edges = true_edges
#     self.train_edges = edges
    self.edge_index = true_edges['edge_index']
    self.edge_reltype = true_edges['edge_reltype']
    self.num_nodes = true_edges['num_nodes']
    self.num_rels = 5
    self.rel_dict = {}
    self.true_edge_dict = {}
    self.filter = filter

    # We construct a dictionary that maps edges to relation types
    # We do this to quickly filter out postive edges while sampling negative edges.
    for i in range(self.true_edges['edge_index'].shape[1]):
      h = self.true_edges['edge_index'][0, i]
      t = self.true_edges['edge_index'][1, i]
      r = self.true_edges['edge_reltype'][i]
      if (h,t) not in self.true_edge_dict:
        self.true_edge_dict[(h,t)] = []
      self.true_edge_dict[(h,t)].append(r)

  def __len__(self):
    return self.edge_index.size(1)

  def _sample_negative_edge(self, idx):
    sample = random.uniform(0, 1)
    found = False
    while not found:
      if sample <= 0.4:
        # corrupt the head entity
        h = self.edge_index[0, idx]
        t = torch.randint(0, self.num_nodes, (1,))
        r = self.edge_reltype[idx]
      elif 0.4 < sample < 0.8:
        # corrupt the tail entity
        t = self.edge_index[1, idx]
        h = torch.randint(0, self.num_nodes, (1,))
        r = self.edge_reltype[idx]
      else:
        # corrupt the relation
        # adding this auxilliary loss is shown to improve performance
        t = self.edge_index[1, idx]
        h = self.edge_index[0, idx]
        r = torch.randint(1269, self.num_rels + 1269 , (1,))
      if not self.filter:
        found = True
      else:
        # check if the edge is a true edge
        if (h, t) not in self.true_edge_dict:
          found = True
        elif r not in self.true_edge_dict[(h, t)]:
          found = True
    data = [torch.tensor([h,t]), r]
    return data

  def __getitem__(self, idx):
    pos_sample = [self.edge_index[:, idx], self.edge_reltype[idx]]
    neg_sample = self._sample_negative_edge(idx)
    return pos_sample, neg_sample

In [10]:
class TestRelationDataset(Dataset):
    def __init__(self, true_edges, filter=False, num_neg=14000, mode='head'):
        self.true_edges = true_edges
        #     self.train_edges = edges
        self.edge_index = true_edges['edge_index']
        self.edge_reltype = true_edges['edge_reltype']
        self.num_nodes = true_edges['num_nodes']
        self.num_rels = 5
        self.rel_dict = {}
        self.true_edge_dict = {}
        self.filter = filter

        # We construct a dictionary that maps edges to relation types
        # We do this to quickly filter out postive edges while sampling negative edges.
        for i in range(self.true_edges['edge_index'].shape[1]):
            h = self.true_edges['edge_index'][0, i]
            t = self.true_edges['edge_index'][1, i]
            r = self.true_edges['edge_reltype'][i]
            if (h,t) not in self.true_edge_dict:
                self.true_edge_dict[(h,t)] = []
            self.true_edge_dict[(h,t)].append(r)
    
    def __len__(self):
         return self.edge_index.size(1)

    def _sample_negative_edge(self, idx, mode):
        triples = []
        random_node_idx = list(range(self.num_nodes))
        random.shuffle(random_node_idx)
        for n in random_node_idx:
            r = self.edge_reltype[idx]
            if mode == 'head':
                # corrupt tail if in head mode
                t = torch.tensor(n)
                h = self.edge_index[0, idx]
            elif mode == 'tail':
                # corrupt head if in tail mode
                h = torch.tensor(n)
                t = self.edge_index[1, idx]
            ht = torch.tensor([h, t])
            if self.filter:
                # check if edge is present in the knowledge graph
                if (h, t) not in self.true_edge_dict:
                    triples.append([ht, r])
                elif r not in self.true_edge_dict[(h, t)]:
                    triples.append([ht, r])
            else:
                triples.append([ht, r])
            if len(triples) == self.num_neg:
                break
        return triples

    def __getitem__(self, idx):
        pos_sample = [self.edge_index[:, idx], self.edge_reltype[idx]]
        neg_samples = self._sample_negative_edge(idx, mode=self.mode)
        edges = torch.stack([pos_sample[0]] + [ht for ht, _ in neg_samples])
        edge_reltype = torch.stack([pos_sample[1]] + [r for _, r in neg_samples])
        return edges, edge_reltype

# Let's define the model

In [11]:
import torch
import torch.nn as nn

class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(TransE, self).__init__()
        self.entity_embeddings = nn.Parameter(torch.randn(num_entities, embedding_dim))
        self.relation_embeddings = nn.Parameter(torch.randn(num_relations, embedding_dim))
    
    def forward(self):
        self.entity_embeddings.data[:-1,:].div_(
        self.entity_embeddings.data[:-1,:].norm(p=2, dim=1, keepdim=True))
        return self.entity_embeddings, self.relation_embeddings

In [12]:
def TransE_loss(pos_edges, neg_edges, pos_reltype, neg_reltype, entity_embeddings, relation_embeddings):
    # Select embeddings for both positive and negative samples
    pos_head_embeds = torch.index_select(entity_embeddings, 0, pos_edges[:, 0])
    pos_tail_embeds = torch.index_select(entity_embeddings, 0, pos_edges[:, 1])
    neg_head_embeds = torch.index_select(entity_embeddings, 0, neg_edges[:, 0])
    neg_tail_embeds = torch.index_select(entity_embeddings, 0, neg_edges[:, 1])
    pos_relation_embeds = torch.index_select(relation_embeddings, 0, pos_reltype.squeeze())
    neg_relation_embeds = torch.index_select(relation_embeddings, 0, neg_reltype.squeeze())

    # Calculate the distance score
    d_pos = torch.norm(pos_head_embeds + pos_relation_embeds - pos_tail_embeds, p=1, dim=1)
    d_neg = torch.norm(neg_head_embeds + neg_relation_embeds - neg_tail_embeds, p=1, dim=1)
    ones = torch.ones(d_pos.size(0))

    # margin loss - we want to increase d_neg and decrease d_pos
    margin_loss = torch.nn.MarginRankingLoss(margin=1.)
    loss = margin_loss(d_neg, d_pos, ones)

    return loss

# Let's create evaluation metric and evaluation function

In [13]:
def eval_metrics(y_pred):
    argsort = torch.argsort(y_pred, dim = 1, descending = False)
    # not using argsort to do the rankings to avoid bias when the scores are equal
    ranking_list = torch.nonzero(argsort == 0, as_tuple=False)
    ranking_list = ranking_list[:, 1] + 1
    hits1_list = (ranking_list <= 1).to(torch.float)
    hits3_list = (ranking_list <= 3).to(torch.float)
    hits10_list = (ranking_list <= 10).to(torch.float)
    mr_list = ranking_list.to(torch.float)
    mrr_list = 1./ranking_list.to(torch.float)
    return hits1_list.mean(), hits3_list.mean(), hits10_list.mean(), mr_list.mean(), mrr_list.mean()

In [14]:
def eval(entity_embeddings, relation_embeddings, dataloader, kg_model = "TransE", iters=None, gamma = 5.0, epsilon = 2.0):
    hits1_list = []
    hits3_list = []
    hits10_list = []
    mr_list = []
    mrr_list = []
    data_iterator = iter(dataloader)
    if iters is None:
        iters = len(dataloader)
    for _ in tqdm.trange(iters, desc="Evaluating"):
        batch = next(data_iterator)
        edges, edge_reltype = batch
        b, num_samples, _= edges.size()
        edges = edges.view(b*num_samples, -1)
        edge_reltype = edge_reltype.view(b*num_samples, -1)

        head_embeds = torch.index_select(entity_embeddings, 0, edges[:, 0])
        relation_embeds = torch.index_select(relation_embeddings, 0, edge_reltype.squeeze())
        tail_embeds = torch.index_select(entity_embeddings, 0, edges[:, 1])

        if kg_model == "TransE":
            scores = torch.norm(head_embeds + relation_embeds - tail_embeds, p=1, dim=1)
        else:
            raise ValueError(f'Unsupported model {kg_model}')

        scores = scores.view(b, num_samples)
    
        hits1, hits3, hits10, mr, mrr = eval_metrics(scores)
        hits1_list.append(hits1.item())
        hits3_list.append(hits3.item())
        hits10_list.append(hits10.item())
        mr_list.append(mr.item())
        mrr_list.append(mrr.item()) 

    hits1 = sum(hits1_list)/len(hits1_list)
    hits3 = sum(hits3_list)/len(hits1_list)
    hits10 = sum(hits10_list)/len(hits1_list)
    mr = sum(mr_list)/len(hits1_list)
    mrr = sum(mrr_list)/len(hits1_list)

    return hits1, hits3, hits10, mr, mrr

# Let's create the model instance

In [15]:
num_entities = 1268
num_relations = 5
embedding_dim = 100

kg_model = "TransE" 
epochs = 100 
batch_size = 128 
learning_rate = 1e-3 

model = TransE(num_entities, num_relations, embedding_dim)
model_loss = TransE_loss

In [16]:
model.entity_embeddings

Parameter containing:
tensor([[ 1.4684,  0.2276,  0.1232,  ..., -0.8076,  1.1650, -0.8651],
        [ 0.5769, -0.7604, -0.1334,  ..., -1.3173,  0.0216, -0.3910],
        [-1.1355, -0.1643,  0.2844,  ..., -0.1874, -1.2541, -1.0203],
        ...,
        [-1.5833, -1.1031,  1.1048,  ..., -1.5604, -0.0812, -0.8178],
        [ 0.8155,  0.2081,  0.9863,  ...,  0.0092,  2.2581, -2.1244],
        [-0.1007, -0.3710, -0.1092,  ...,  0.8020,  0.0508, -0.5738]],
       requires_grad=True)

In [21]:
import os
num_workers = os.cpu_count()

train_dataset = RelationDataset(train_set, filter = True)
train_dataloader = DataLoader(train_dataset, batch_size= batch_size, shuffle=True, num_workers=num_workers)
val_dataset = RelationDataset(val_set, filter = True)
val_dataloader = DataLoader(val_dataset, batch_size= batch_size, shuffle=False, num_workers=num_workers)

val_eval_dataset = TestRelationDataset(val_set, filter = True, num_neg= 100)
val_eval_dataloader = DataLoader(val_eval_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_dataset = TestRelationDataset(test_set, filter=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f'Train dataset size {len(train_dataset)}')
print(f'Val dataset size {len(val_dataset)}')
print(f'Val dataset size {len(val_eval_dataset)}')
print(f'Val dataset size {len(test_dataset)}')

Train dataset size 9155
Val dataset size 3051
Val dataset size 3051
Val dataset size 3053


# Let's train the model

In [ ]:
# We use adam optimizer for training
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

for e in range(epochs):
    losses = []
    #check metric for every 10th epoch
    if e%10 == 0:
        model.eval()
        h1, h3, h10, mr, mrr = eval(model.entity_embeddings, model.relation_embeddings, val_eval_dataloader, kg_model, iters=10)
        print(f"hits@1:{h1} hits@3:{h3} hits@10:{h10} mr:{mr} mrr:{mrr}")
    model.train()
    for step, batch in enumerate(tqdm.tqdm(train_dataloader, desc="Training")):
        pos_sample, neg_sample = batch
        entity_embedding_sample, relation_embedding_sample = model()
        optimizer.zero_grad()
        
        loss = model_loss(pos_sample[0], neg_sample[0], pos_sample[1], neg_sample[1], entity_embedding_sample, relation_embedding_sample)
        
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    
    val_losses = []
    model.eval()
    entity_embedding_pass, relation_embedding_pass = model()
    for step, batch in enumerate(tqdm.tqdm(val_dataloader, desc="Validation")):
        pos_sample, neg_sample = batch
        loss = model_loss(pos_sample[0], neg_sample[0], pos_sample[1], neg_sample[1], entity_embedding_pass, relation_embedding_pass)
        val_losses.append(loss.item())
    print(f"epoch: {e + 1} loss: {sum(losses)/len(losses)} val_loss: {sum(val_losses)/len(val_losses)}")

In [ ]:
a, b = train_dataloader
a[0]